# Turbofan Tutorial
`pyturbo`  library is provided by twiinIT to assembly a simple turbofan system.

The library is made of components: 

- `compressor` : fluid out is computed from fluid in and power provided by shaft with constant efficiency. 
- `combustor` : combustion is made considering constant FHV.
- `turbine` : power is extracted from fluid in considering a given expansion ratio and constant efficiency. 
- `inlet` and `nozzle` are computing `drag` and `thrust` from fluid conditions, ambiant pressure and throat section. 
- `nacelle`: envelop over the engine.
- `ogv`, `intermediate_casing` and `trf` and structures with aero channels. 

They are numerical components:

- `fluid_spitter` is used to split the flow into primary and secondary flow
- `shaft_spitter` is used to split the shaft power into booster and fan compressor

Aero 0D and simplified geometry are considered. 

A turbofan system is generated.

In [1]:
from pyturbo.systems.turbofan import TurbofanWithAtm
sys = TurbofanWithAtm("sys")
tf = sys.tf

In [ ]:
sys

In [2]:
# geometrical view
sys.run_once()

from pyturbo.utils import jupyter_view

jupyter_view(tf).show()

Renderer(background='pink', camera=PerspectiveCamera(aspect=2.25, children=(DirectionalLight(intensity=0.3, po…

# Simulation

The turbofan system has a couple of equations/unknowns to solve. We use `cosapp` non-linear solver for this purpose.

In [3]:
from cosapp.drivers import NonLinearSolver

## Direct mode
`thrust` is computed from `fuel_W`.

In [4]:
# off-design solver
run = sys.add_driver(NonLinearSolver('run', tol = 1e-6))

In [5]:
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

# use case
tf.fuel_W = .5

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', tf.pamb, 'Pa')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')

mach = 0.0
pamb = 101325.0 Pa
thrust = 19.0 klbf
thrust = 84 kN
N1 = 6577 rpm
N2 = 23444 rpm
bpr = 7.5
opr = 34.1
T41 = 1786 K
sfc = 0.426 kg/(h*kN)


In [6]:
run.problem

Unknowns [11]
  tf.fl_in.W = 442.3108771334678
  tf.fan_module.splitter_shaft.power_fractions = [0.66799579]
  tf.fan_module.splitter_fluid.fluid_fractions = [0.88187347]
  tf.core.turbine.aero.Ncqdes = 145.25050180808654
  tf.core.turbine.aero.dhqt = 241.11044787544756
  tf.core.compressor.sh_in.power = 22924263.2116414
  tf.core.compressor.sh_in.N = 23444.14477842246
  tf.turbine.aero.Ncqdes = 124.65263121792852
  tf.turbine.aero.dhqt = 415.23336786508804
  tf.fan_module.sh_in.power = 29998501.196282748
  tf.fan_module.sh_in.N = 6576.782027006616
Equations [11]
  tf.fan_module.fan.aero: eps_psi == 0 := 4.440892098500626e-16
  tf.fan_module.booster.aero: eps_psi == 0 := -7.105427357601002e-15
  tf.core.compressor.aero: eps_psi == 0 := -5.0681681074138396e-14
  tf.core.turbine.aero: fl_in.W == Wcrit := 4.050093593832571e-13
  tf.core: compressor.sh_in.power == turbine.sh_out.power (loop) := -7.450580596923828e-09
  tf.core: compressor.sh_in.N == turbine.sh_out.N (loop) := -3.3469405025

## control mode
`fuel_W` is computed to match functional request, here the fan rotational speed `N1` value.

In [8]:
# control solver
sys.drivers.clear()
run = sys.add_driver(NonLinearSolver('run', tol=1e-6))
run.add_unknown('tf.fuel_W')
run.add_target('tf.N1')

Unknowns [1]
  tf.fuel_W = 1.0
Equations [1]
  tf.N1 == 6576.782027006616 (target)

In [9]:
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

# use case
tf.N1 = 5000.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')

mach = 0.0
pamb = 101325.0 Pa
thrust = 10.9 klbf
thrust = 48 kN
N1 = 5000 rpm
N2 = 20465 rpm
bpr = 8.9
opr = 19.2
T41 = 1430 K
sfc = 0.358 kg/(h*kN)


In [10]:
run.problem

Unknowns [12]
  fuel_W = 0.4811320909299096
  tf.fl_in.W = 328.08656070504475
  tf.fan_module.splitter_shaft.power_fractions = [0.68751138]
  tf.fan_module.splitter_fluid.fluid_fractions = [0.89946144]
  tf.core.turbine.aero.Ncqdes = 141.70601632228133
  tf.core.turbine.aero.dhqt = 241.1104478754474
  tf.core.compressor.sh_in.power = 11535023.640473248
  tf.core.compressor.sh_in.N = 20465.193829989716
  tf.turbine.aero.Ncqdes = 105.91249192847229
  tf.turbine.aero.dhqt = 358.980195330147
  tf.fan_module.sh_in.power = 13049711.277447995
  tf.fan_module.sh_in.N = 5000.0
Equations [12]
  tf.N1 == 5000.0 := 0.0
  tf.fan_module.fan.aero: eps_psi == 0 := -1.942890293094024e-16
  tf.fan_module.booster.aero: eps_psi == 0 := -2.220446049250313e-16
  tf.core.compressor.aero: eps_psi == 0 := 1.27675647831893e-15
  tf.core.turbine.aero: fl_in.W == Wcrit := 1.0658141036401503e-13
  tf.core: compressor.sh_in.power == turbine.sh_out.power (loop) := -1.862645149230957e-09
  tf.core: compressor.sh_in.N

## design mode

Turbofan design characteristics are related to components and physical properties.

### Using design methods

In [14]:
# design solver
sys.drivers.clear()
design = sys.add_driver(NonLinearSolver('design', tol=1e-6, factor = 0.2))
design.add_unknown('tf.fuel_W')
design.add_target('tf.thrust')

# init
tf.fl_in.W = 300.

# engine
sys.thrust = 120e3
tf.bpr = 5.
tf.pr_nozzle = 1.1

# inlet
tf.inlet.aero.mach = 0.5

# fan module
tf.fan_module.fan.aero.pcnr = 95.
tf.fan_module.fan.aero.utip = 420.

# booster
tf.fan_module.booster.aero.phi = 0.45
tf.fan_module.booster.aero.psi = 0.35
tf.fan_module.booster.aero.spec_flow = 180.
tf.fan_module.booster.aero.pcnr = 95.

# lpt
tf.turbine.aero.Ncqdes = 100.
tf.turbine.aero.psi = 1.25

# hpc
tf.core.compressor.aero.pr = 11.
tf.core.compressor.aero.utip = 420.
tf.core.compressor.aero.phi = 0.5
tf.core.compressor.aero.pcnr = 95.

# hpt
tf.core.turbine.aero.psi = 1.2
tf.core.turbine.aero.Ncqdes = 100.

# combustor
tf.core.combustor.aero.Tcomb = 1700.

# design method (using targets)
design.extend(tf.design_methods['scaling'])

Unknowns [19]
  tf.fuel_W = 0.4811320909299096
  tf.fan_diameter = 1.6
  tf.fan_module.fan.aero.xnd = 10000.0
  tf.fan_module.fan.aero.phiP = 0.4
  tf.fan_module.geom.booster_radius_ratio = 0.6
  tf.fan_module.booster.geom.blade_hub_to_tip_ratio = 0.6
  tf.fan_module.booster.aero.phiP = 0.7
  tf.fan_module.booster.aero.xnd = 10000.0
  tf.geom.turbine_radius_ratio = 0.65
  tf.turbine.geom.blade_height_ratio = 0.4
  tf.turbine.aero.Ncdes = 15.0
  tf.geom.core_inlet_radius_ratio = 0.25
  tf.core.compressor.aero.xnd = 10000.0
  tf.core.compressor.aero.phiP = 0.9
  tf.geom.core_exit_radius_ratio = 0.3
  tf.core.turbine.geom.blade_height_ratio = 0.2
  tf.core.turbine.aero.Ncdes = 40.0
  tf.geom.pri_nozzle_area_ratio = 0.9
  tf.geom.sec_nozzle_area_ratio = 0.9
Equations [19]
  tf.thrust == 48377.41103005884 (target)
  tf.inlet.aero.mach == 0.5 (target)
  tf.fan_module.fan.aero.pcnr == 95.0 (target)
  tf.fan_module.fan.aero.utip == 420.0 (target)
  tf.bpr == 5.0 (target)
  tf.fan_module.booste

In [15]:
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fan diameter =', round(tf.geom.fan_diameter / 0.0254, 1), 'in')
print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

mach = 0.0
pamb = 101325.0 Pa
fan diameter = 70.7 in
fan diameter = 1.8 m
W = 445 Kg/s
thrust = 27.0 klbf
thrust = 120 kN
N1 = 4464 rpm
N2 = 10947 rpm
bpr = 5.0
opr = 30.8
T41 = 1700 K
sfc = 0.398 kg/(h*kN)
psi fan = 0.36
psi booster = 0.35
psi hpc = 0.3


In [16]:
design.problem

Unknowns [30]
  fuel_W = 1.3250536826971522
  tf.fan_diameter = 1.7968023771596553
  tf.fan_module.fan.aero.xnd = 4699.230514804913
  tf.fan_module.fan.aero.phiP = 0.5148161480335711
  tf.fan_module.geom.booster_radius_ratio = 0.7974470236685992
  tf.fan_module.booster.geom.blade_hub_to_tip_ratio = 0.8589597363714984
  tf.fan_module.booster.aero.phiP = 0.6923076923076923
  tf.fan_module.booster.aero.xnd = 4699.230514804913
  tf.geom.turbine_radius_ratio = 0.44993051805210593
  tf.turbine.geom.blade_height_ratio = 0.22677912081967153
  tf.turbine.aero.Ncdes = 13.339504963887562
  tf.geom.core_inlet_radius_ratio = 0.4077925594662063
  tf.core.compressor.aero.xnd = 11523.580815099043
  tf.core.compressor.aero.phiP = 0.7180590258000786
  tf.geom.core_exit_radius_ratio = 0.45413239875331163
  tf.core.turbine.geom.blade_height_ratio = 0.10038561179321587
  tf.core.turbine.aero.Ncdes = 27.80450798744951
  tf.geom.pri_nozzle_area_ratio = 1.7954804169358394
  tf.geom.sec_nozzle_area_ratio = 0.6

### Using raw equations/unknowns (detailed)

In [18]:
# design mode solver
sys.drivers.clear()
design = sys.add_driver(NonLinearSolver('design', tol=1e-6, factor=0.2))

# engine
sys.thrust = 120e3
tf.bpr = 5.
tf.pr_nozzle = 1.1

# inlet
tf.inlet.aero.mach = 0.5

# fan module
tf.fan_module.fan.aero.utip = 420.
tf.fan_module.fan.aero.pcnr = 95.

# booster
tf.fan_module.booster.aero.phi = 0.45
tf.fan_module.booster.aero.psi = 0.35
tf.fan_module.booster.aero.spec_flow = 180.
tf.fan_module.booster.aero.pcnr = 95.

# lpt
tf.turbine.aero.Ncqdes = 100.
tf.turbine.aero.psi = 1.25

# hpc
tf.core.compressor.aero.pr = 11.
tf.core.compressor.aero.utip = 420.
tf.core.compressor.aero.phi = 0.5
tf.core.compressor.aero.pcnr = 95.

# hpt
tf.core.turbine.aero.psi = 1.2
tf.core.turbine.aero.Ncqdes = 100.

# combustor
tf.core.combustor.aero.Tcomb = 1700.

# engine
design.add_unknown('tf.fuel_W')
design.add_target('tf.thrust')
design.add_unknown('tf.fan_diameter')

# inlet
design.add_target('tf.inlet.aero.mach')

# fan
design.add_unknown("tf.fan_module.fan.aero.xnd", max_rel_step=0.5)
design.add_unknown('tf.fan_module.fan.aero.phiP', lower_bound=0.1, upper_bound=1.5)

design.add_target("tf.fan_module.fan.aero.pcnr")
design.add_target('tf.fan_module.fan.aero.utip')
design.add_target('tf.bpr')

# booster
design.add_unknown('tf.fan_module.geom.booster_radius_ratio')
design.add_unknown('tf.fan_module.booster.geom.blade_hub_to_tip_ratio', lower_bound=1e-5, upper_bound=1.)
design.add_unknown('tf.fan_module.booster.aero.phiP')
design.add_unknown("tf.fan_module.booster.aero.xnd", max_rel_step=0.5)

design.add_target('tf.fan_module.booster.aero.phi')
design.add_target('tf.fan_module.booster.aero.psi')
design.add_target('tf.fan_module.booster.aero.spec_flow')
design.add_target("tf.fan_module.booster.aero.pcnr")

# lpt
design.add_unknown('tf.geom.turbine_radius_ratio')
design.add_unknown("tf.turbine.geom.blade_height_ratio", lower_bound=0., upper_bound=1.)
design.add_unknown('tf.turbine.aero.Ncdes')

design.add_target('tf.turbine.aero.psi')
design.add_target('tf.turbine.aero.Ncqdes')

# hpc
design.add_unknown('tf.geom.core_inlet_radius_ratio', max_rel_step=0.8)
design.add_unknown("tf.core.compressor.aero.xnd", max_rel_step=0.5)
design.add_unknown("tf.core.compressor.aero.phiP")

design.add_target("tf.core.compressor.aero.pcnr")
design.add_target("tf.core.compressor.aero.phi")
design.add_target("tf.core.compressor.aero.utip")
design.add_target("tf.core.compressor.aero.pr")

# combustor
design.add_target("tf.core.combustor.aero.Tcomb")

# hpt
design.add_unknown('tf.geom.core_exit_radius_ratio', max_rel_step=0.8)
design.add_unknown("tf.core.turbine.geom.blade_height_ratio", lower_bound=0., upper_bound=1.)
design.add_unknown("tf.core.turbine.aero.Ncdes")

design.add_target("tf.core.turbine.aero.psi")
design.add_target("tf.core.turbine.aero.Ncqdes")

# nozzle
design.add_unknown('tf.geom.pri_nozzle_area_ratio', lower_bound=0.05)
design.add_unknown('tf.geom.sec_nozzle_area_ratio', upper_bound=1.)

design.add_target('tf.pr_nozzle')

Unknowns [19]
  tf.fuel_W = 1.3250536826971522
  tf.fan_diameter = 1.7968023771596553
  tf.fan_module.fan.aero.xnd = 4699.230514804913
  tf.fan_module.fan.aero.phiP = 0.5148161480335711
  tf.fan_module.geom.booster_radius_ratio = 0.7974470236685992
  tf.fan_module.booster.geom.blade_hub_to_tip_ratio = 0.8589597363714984
  tf.fan_module.booster.aero.phiP = 0.6923076923076923
  tf.fan_module.booster.aero.xnd = 4699.230514804913
  tf.geom.turbine_radius_ratio = 0.44993051805210593
  tf.turbine.geom.blade_height_ratio = 0.22677912081967153
  tf.turbine.aero.Ncdes = 13.339504963887562
  tf.geom.core_inlet_radius_ratio = 0.4077925594662063
  tf.core.compressor.aero.xnd = 11523.580815099043
  tf.core.compressor.aero.phiP = 0.7180590258000786
  tf.geom.core_exit_radius_ratio = 0.45413239875331163
  tf.core.turbine.geom.blade_height_ratio = 0.10038561179321587
  tf.core.turbine.aero.Ncdes = 27.80450798744951
  tf.geom.pri_nozzle_area_ratio = 1.7954804169358394
  tf.geom.sec_nozzle_area_ratio = 

In [19]:
# environment conditions
sys.altitude = 0.
sys.mach = 0.
sys.dtamb = 15.

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fan diameter =', round(tf.geom.fan_diameter / 0.0254, 1), 'in')
print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

mach = 0.0
pamb = 101325.0 Pa
fan diameter = 70.7 in
fan diameter = 1.8 m
W = 445 Kg/s
thrust = 27.0 klbf
thrust = 120 kN
N1 = 4464 rpm
N2 = 10947 rpm
bpr = 5.0
opr = 30.8
T41 = 1700 K
sfc = 0.398 kg/(h*kN)
psi fan = 0.36
psi booster = 0.35
psi hpc = 0.3


In [20]:
jupyter_view(tf).show()

Renderer(background='pink', camera=PerspectiveCamera(aspect=2.25, children=(DirectionalLight(intensity=0.3, po…

### off-design computation after design

Fuel consumption for a given altitude/mach/dtamb and thrust. 

In [21]:
# off-design mode
sys.drivers.clear()
run = sys.add_driver(NonLinearSolver('run', tol=1e-6))

In [23]:
# environment conditions
sys.altitude = 10000.
sys.mach = 0.8
sys.dtamb = 0.

# requirement
tf.thrust = 10e3 / 0.224809
# tf.N1 = 5000.

run.add_unknown('tf.fuel_W')
run.add_target('tf.thrust')
# run.runner.add_target('tf.N1')

sys.run_drivers()

print('mach =', sys.mach)
print('pamb =', sys.atm.pamb, 'Pa')
print('fuel flow =', round(tf.fuel_W, 1), 'kg/s')
print('W =', round(tf.fl_in.W), 'Kg/s')
print('thrust =', round(tf.thrust * 0.224809/1e3, 1), 'klbf')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")
print('N2 =', round(tf.N2), "rpm")
print('bpr =', round(tf.bpr, 1))
print('opr =', round(tf.opr, 1))
print('T41 =', round(tf.core.turbine.fl_in.Tt), 'K')
print('sfc =', round(tf.sfc, 3), 'kg/(h*kN)')
print('psi fan =', round(tf.fan_module.fan.aero.psi, 2))
print('psi booster =', round(tf.fan_module.booster.aero.psi, 2))
print('psi hpc =', round(tf.core.compressor.aero.psi, 2))

mach = 0.8
pamb = 26499.87312280235 Pa
fuel flow = 2.1 kg/s
W = 352 Kg/s
thrust = 27.0 klbf
thrust = 120 kN
N1 = 6635 rpm
N2 = 16664 rpm
bpr = 4.1
opr = 85.7
T41 = 2353 K
sfc = 0.618 kg/(h*kN)
psi fan = 0.32
psi booster = 0.15
psi hpc = 0.18
